<a href="https://colab.research.google.com/github/mcash8/DVBS2X-DNN-Implementation/blob/main/8APSK_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import scipy.io as spio
import math
import time
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
start_time = time.time()

#parameters
num_classes = 2
batch_size = 64
epochs = 100

%cd /content/drive/My Drive/NN\ Input/AWGN/8APSK
snrs = [2, 10, 23]

for snr in snrs: 
    matname = '8APSK_data_snr_%d.mat' % (snr)
    mat = spio.loadmat(matname, squeeze_me=True)
    x_train = mat['x_train']
    x_valid = mat['val_data']
    y_valid = mat['val_target']
    y_train = mat['y_train']


    #cast to float32
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    #x_test = x_test.astype('float32') 

    print(x_train.shape, 'train samples')
    print(x_valid.shape, 'valid samples')

    print(y_train.shape, 'train labels')
    print(y_valid.shape, 'valid labels')

    print('Label Examples:\n', y_train)
    print('Input Examples:\n', x_train)

    # Define the network
    model = Sequential()
    model.add(Dense(50, activation='elu', input_dim=18))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(num_classes, activation='tanh'))

    model.summary()

    model.compile(loss=keras.metrics.mean_squared_error,
                optimizer=SGD(learning_rate=0.1),
                metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=2,
                        validation_data=(x_valid, y_valid))

    elapsedTime = (time.time() - start_time)/60
    print("--- %.2f minutes ---" % elapsedTime)

    score = model.evaluate(x_train, y_train, verbose=2)
    print('Final Training MSE:', score[0])
    print('Final Training RMSE:', score[1])

    #save model
    savename = '8APSK_SNR_%d.h5' % (snr)
    %cd /content/drive/My Drive/NN\ Input/AWGN/8APSK
    model.save(savename)
    %cd /content/drive/My Drive/NN\ Input/AWGN/8APSK

In [ ]:
#load model and make predictions
i = 0
model_names = ['8APSK_SNR_2.h5','8APSK_SNR_10.h5','8APSK_SNR_23.h5']
snr = list(range(-2,24))

for name in model_names: 
    %cd /content/drive/My Drive/NN\ Input/AWGN/8APSK
    model = tf.keras.models.load_model(name)
    for snrs in snr:
        foldername = 'SNR_%d' % snrs
        %cd /content/drive/My Drive/NN\ Input/AWGN/Test\ Data/8APSK
        %cd $foldername
        for x in range(1,11):
            #load test data
            matname = '8APSK_pred_%d' % (x)
            mat = spio.loadmat(matname, squeeze_me=True)
            x_test = mat['train_data']

            predictions = model.predict(x_test)

            #save predictions
            matname = '8APSK_out_snr_%d_%d_mdl_%d.mat' % (snrs, x, i)
            spio.savemat(matname, {'pred': predictions})
    i+=1; 